## Check Drivers and Set Environment Variables

In [1]:
import os

os.environ["SPARK_HOME"] = f"{os.getcwd()}/pyspark/spark-3.4.4-bin-hadoop3"
os.environ["PATH"] =f"{os.environ.get('SPARK_HOME')}/bin:{os.environ.get('PATH')}"
os.environ["SPARK_JARS"] = '/media/HDD/JeYoung/Fundamentals_of_GPU_DataScience_with_RAPIDS/Chapter07/pyspark/rapids-4-spark_2.12-24.12.0.jar'


os.environ["PYSPARK_PYTHON"] = "python3"
os.environ["PYSPARK_DRIVER_PYTHON"] = "jupyter"
os.environ["PYSPARK_DRIVER_PYTHON_OPTS"] = "notebook"

# Set CUDA Visiblity
os.environ["CUDA_VISIBLE_DEVICES"] = "0" 
# If you use port 127.0.0.1(basic port), you don't need to set below codes
os.environ["SPARK_LOCAL_IP"] = "192.168.0.69"


# if you need to add JAVA path, run below code
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
# os.environ["PATH"]=f"{os.environ.get('JAVA_HOME')}/bin:{os.environ.get('PATH')}"

!echo $SPARK_HOME
!echo $PWD
!echo $SPARK_JARS

/media/HDD/JeYoung/Fundamentals_of_GPU_DataScience_with_RAPIDS/Chapter07/pyspark/spark-3.4.4-bin-hadoop3
/media/HDD/JeYoung/Fundamentals_of_GPU_DataScience_with_RAPIDS/Chapter07
/media/HDD/JeYoung/Fundamentals_of_GPU_DataScience_with_RAPIDS/Chapter07/pyspark/rapids-4-spark_2.12-24.12.0.jar


In [2]:
# 1. Java 확인
!java -version
print("JAVA_HOME:", os.environ.get("JAVA_HOME"))

# 2. PySpark 확인
!pip show pyspark

# 3. Spark 홈 디렉토리 확인
print("SPARK_HOME:", os.environ.get("SPARK_HOME"))

# 4. Python 실행 경로 확인
!which python
print("PYSPARK_PYTHON:", os.environ.get("PYSPARK_PYTHON"))

openjdk version "11.0.26" 2025-01-21
OpenJDK Runtime Environment (build 11.0.26+4-post-Ubuntu-1ubuntu120.04)
OpenJDK 64-Bit Server VM (build 11.0.26+4-post-Ubuntu-1ubuntu120.04, mixed mode, sharing)
JAVA_HOME: /usr/lib/jvm/java-11-openjdk-amd64
Name: pyspark
Version: 3.4.4
Summary: Apache Spark Python API
Home-page: https://github.com/apache/spark/tree/master/python
Author: Spark Developers
Author-email: dev@spark.apache.org
License: http://www.apache.org/licenses/LICENSE-2.0
Location: /home/cvmi_jeyoung/anaconda3/envs/rapids2/lib/python3.10/site-packages
Requires: py4j
Required-by: 
SPARK_HOME: /media/HDD/JeYoung/Fundamentals_of_GPU_DataScience_with_RAPIDS/Chapter07/pyspark/spark-3.4.4-bin-hadoop3
/home/cvmi_jeyoung/anaconda3/envs/rapids2/bin/python
PYSPARK_PYTHON: python3


In [3]:
!nvidia-smi

Thu Mar  6 09:13:49 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 2080 ...    Off |   00000000:05:00.0 Off |                  N/A |
|  0%   47C    P8             13W /  250W |      14MiB /   8192MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## Set Optimized Pyspark RAPIDS Enviroments
---
You can configure PySpark settings through environment variables.

In [4]:
os.environ["PYSPARK_SUBMIT_ARGS"] = (
    f'--jars {os.environ.get("SPARK_JARS")} '
    "--master local[*] "
    "--conf spark.plugins=com.nvidia.spark.SQLPlugin "
    "--conf spark.rapids.sql.enabled=true "
    "--conf spark.rapids.sql.explain=NONE "  # GPU 실행 로그를 출력하도록 설정
    "--conf spark.rapids.memory.gpu.allocFraction=0.6 "  # GPU 메모리 사용량 60% 설정
    "--conf spark.rapids.memory.gpu.maxAllocFraction=0.9 "  # 최대 90%까지 사용 가능
    "pyspark-shell"    
)


In [5]:
os.environ.get("PYSPARK_SUBMIT_ARGS")

'--jars /media/HDD/JeYoung/Fundamentals_of_GPU_DataScience_with_RAPIDS/Chapter07/pyspark/rapids-4-spark_2.12-24.12.0.jar --master local[*] --conf spark.plugins=com.nvidia.spark.SQLPlugin --conf spark.rapids.sql.enabled=true --conf spark.rapids.sql.explain=NONE --conf spark.rapids.memory.gpu.allocFraction=0.6 --conf spark.rapids.memory.gpu.maxAllocFraction=0.9 pyspark-shell'

## Start jupyter notebook with plugin config

In [6]:
import time
import os
import cupy as cp
import pyspark
from pyspark import broadcast
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

import findspark
findspark.init()

# Check Spark Version
print(pyspark.__version__)

3.4.4


In [7]:
spark = (SparkSession.builder
         .appName("Test GPU")
        .getOrCreate())

spark


25/03/06 09:13:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/06 09:13:52 WARN RapidsPluginUtils: RAPIDS Accelerator 24.12.0 using cudf 24.12.0, private revision 874fe2a69931879dc50faad111280e8ca75467cc
25/03/06 09:13:52 WARN RapidsPluginUtils: RAPIDS Accelerator is enabled, to disable GPU support set `spark.rapids.sql.enabled` to false.
25/03/06 09:14:02 ERROR Inbox: Ignoring error
java.lang.NullPointerException
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$register(BlockManagerMasterEndpoint.scala:600)
	at org.apache.spark.storage.BlockManagerMasterEndpoint$$anonfun$receiveAndReply$1.applyOrElse(BlockManagerMasterEndpoint.scala:123)
	at org.apache.spark.rpc.netty.Inbox.$anonfun$process$1(Inbox.scala:103)
	at 

In [8]:
df = spark.range(10000).toDF("num")
df.selectExpr("num * 2 as double_num").show()

+----------+
|double_num|
+----------+
|         0|
|         2|
|         4|
|         6|
|         8|
|        10|
|        12|
|        14|
|        16|
|        18|
|        20|
|        22|
|        24|
|        26|
|        28|
|        30|
|        32|
|        34|
|        36|
|        38|
+----------+
only showing top 20 rows



In [9]:
#init spark
spark.sparkContext.cancelAllJobs()
spark.stop()

## Change Configuration on Codes
---

 If you want to optimize your PySpark code, 
 
 Try customizing the configuration in your code to improve performance.

 See The Configuration Doc [LINK](https://nvidia.github.io/spark-rapids/docs/configs.html)




In [10]:
spark = (SparkSession.builder.appName("Test GPU")
        .master("local-cluster[2,8,8192]") # 2 Executor, 8 Core, 8192MB per Executor
        .getOrCreate())

spark


25/03/06 09:14:08 WARN RapidsPluginUtils: RAPIDS Accelerator is enabled, to disable GPU support set `spark.rapids.sql.enabled` to false.


In [11]:
df = spark.range(10000).toDF("num")
df.selectExpr("num * 2 as double_num").show()

[Stage 0:>                                                          (0 + 1) / 1]

+----------+
|double_num|
+----------+
|         0|
|         2|
|         4|
|         6|
|         8|
|        10|
|        12|
|        14|
|        16|
|        18|
|        20|
|        22|
|        24|
|        26|
|        28|
|        30|
|        32|
|        34|
|        36|
|        38|
+----------+
only showing top 20 rows



If you want to see other option

See [LINK](https://nvidia.github.io/spark-rapids/docs/additional-functionality/advanced_configs.html)

In [28]:
#init spark
spark.sparkContext.cancelAllJobs()
spark.stop()

In [29]:
spark = (SparkSession.builder.appName("Test GPU")
         # .master("local-cluster[2,8,8192]") # 2 Excutor, 4 Core, 2048MB         
        # .config("spark.dynamicAllocation.enabled", "false")
        # .config("spark.rapids.sql.enabled", "true")
         .config("spark.executor.resource.gpu.discoveryScript", "./sample_scripts/GPU_info.sh")
         .config("spark.executor.instances", "2")  # Executor 개수 줄이기
         .config("spark.executor.cores", "2")
         .config("spark.executor.resource.gpu.amount", "1")
        .config("spark.task.resource.gpu.amount", "0.5") 
        # .config("spark.executor.resource.gpu.amount", "1")   # Executor당 1개 GPU 사용
         .config("spark.executorEnv.CUDA_VISIBLE_DEVICES", "0,1") # For Multi-GPUs

        # .config("spark.rapids.memory.gpu.allocFraction", "0.6") 
        # .config("spark.rapids.memory.gpu.maxAllocFraction", "0.9") 
         
        # .config("spark.plugins", "com.nvidia.spark.SQLPlugin")
        # .config("spark.jars", os.environ.get('SPARK_JARS')) 
        .config("spark.rapids.sql.multiThreadedRead.numThreads", "10")
        .config('spark.rapids.sql.enabled','true')
        .config('spark.rapids.sql.format.csv.read.enabled', 'true')
        .config('spark.rapids.sql.format.csv.enabled', 'true')
         # .config('spark.dynamicAllocation.enabled', 'false')
        # .config("spark.sql.autoBroadcastJoinThreshold", "-1")
        .getOrCreate())

spark


25/03/06 14:47:00 WARN RapidsPluginUtils: RAPIDS Accelerator is enabled, to disable GPU support set `spark.rapids.sql.enabled` to false.
25/03/06 14:47:02 WARN GpuDeviceManager: The default cuDF host pool was already configured


In [ ]:
df = spark.range(100000).toDF("num")
df.selectExpr("num * 2 as double_num").show()

[Stage 0:>                                                          (0 + 0) / 1]

In [ ]:
#init spark
spark.sparkContext.cancelAllJobs()
spark.stop()

In [ ]:
spark = (SparkSession.builder.appName("Test GPU")
         .master("local-cluster[2,8,8192]") # 2 Excutor, 4 Core, 2048MB         
        .config("spark.dynamicAllocation.enabled", "false")
        # .config("spark.rapids.sql.enabled", "true")
        .config("spark.executor.resource.gpu.amount", "1")
        .config("spark.task.resource.gpu.amount", "0.25") 

        # .config("spark.rapids.memory.gpu.allocFraction", "0.6") 
        # .config("spark.rapids.memory.gpu.maxAllocFraction", "0.9") 
         
        # .config("spark.plugins", "com.nvidia.spark.SQLPlugin")
        # .config("spark.jars", os.environ.get('SPARK_JARS')) 
        # .config("spark.rapids.sql.multiThreadedRead.numThreads", "10")
        # .config("spark.driver.extraJavaOptions", "-Djava.library.path=/usr/lib/jvm/java-11-openjdk-amd64")
        # .config('spark.rapids.sql.enabled','true')
        # .config('spark.rapids.sql.format.csv.read.enabled', 'true')
        # .config('spark.rapids.sql.format.csv.enabled', 'true')
         # .config('spark.dynamicAllocation.enabled', 'false')
        # .config("spark.sql.autoBroadcastJoinThreshold", "-1")
        .getOrCreate())

spark


In [ ]:
print("spark.plugins:", spark.conf.get("spark.plugins"))
print("spark.rapids.sql.enabled:", spark.conf.get("spark.rapids.sql.enabled"))
# print("spark.executor.cores:", spark.conf.get("spark.executor.cores"))
# print("spark.executor.resource.gpu.amount:", spark.conf.get("spark.executor.resource.gpu.amount"))
# print("spark.task.resource.gpu.amount:", spark.conf.get("spark.task.resource.gpu.amount"))
# print("SparkContext 상태:", spark.sparkContext._jsc.sc().isStopped())
# print("Spark Master URL:", spark.conf.get("spark.master"))

In [ ]:
df = spark.range(100).toDF("num")
df.cache()  # ✅ Spark에 데이터 캐싱 → Task 실행 유도
df.selectExpr("num * 2 as double_num").show()

In [ ]:
#init spark
spark.sparkContext.cancelAllJobs()
spark.stop()

In [ ]:
!nvidia-smi

## Create Spark Session

In [ ]:
spark = (SparkSession
    .builder
    .appName("TitanicETL")
    .getOrCreate())

# spark.sparkContext.setLogLevel("INFO")  # 로그 레벨을 INFO로 변경

In [ ]:
print("spark.plugins:", spark.conf.get("spark.plugins"))
print("spark.rapids.sql.enabled:", spark.conf.get("spark.rapids.sql.enabled"))
print("spark.executor.cores:", spark.conf.get("spark.executor.cores"))
print("spark.executor.resource.gpu.amount:", spark.conf.get("spark.executor.resource.gpu.amount"))
print("spark.task.resource.gpu.amount:", spark.conf.get("spark.task.resource.gpu.amount"))
print("SparkContext 상태:", spark.sparkContext._jsc.sc().isStopped())



In [ ]:
spark.sparkContext.uiWebUrl  # Spark UI 주소 확인

## GPU Check

In [ ]:
gpu_info = spark.conf.get("spark.executor.resource.gpu.amount", "Not Found")
print(f"✅ Spark가 감지한 GPU 수: {gpu_info}")

In [ ]:
_csv_raw_schema = StructType([
    StructField('PassengerId', IntegerType()),
    StructField("Survival", IntegerType()),
    StructField("Pclass", IntegerType()),
    StructField('Name', StringType()),
    StructField("Sex", StringType()),
    StructField("Age", DoubleType()),
    StructField("SibSp", IntegerType()),
    StructField("Parch", IntegerType()),
    StructField("Ticket ", StringType()),
    StructField("Fare", DoubleType()),
    StructField("Cabin", StringType()),
    StructField("Embarked", StringType()),])
    

In [ ]:
cate_col_names = [
        "Name",
        "Sex",
        "Ticket",
        "Cabin",
        "Embarked",
]

label_col_name = "Survival"
numeric_col_names = [
        "PassengerId",
        "Pclass",
        "Age",
        "SibSp",
        "Parch",
        "Fare",
        label_col_name
]

all_col_names = cate_col_names + numeric_col_names

In [ ]:
def extract_perf_columns(rawDf):
    perfDf = rawDf.select(
        col('Pclass'),
        col('Sex'),
        col('SibSp'),
        col('Parch'),
        col('Age'),
        col('Embarked_l'),
        col('Fare'),
        col('Survival'),
    )

    return perfDf.select("*")

def read_raw_csv(spark, path):
    return spark.read.format('csv') \
            .option('nullValue', cp.nan) \
            .option('delimiter', ',') \
            .schema(_csv_raw_schema) \
            .load(path)
        

In [ ]:
# spark.conf.set('spark.rapids.sql.explain', 'ALL')
# spark.conf.set('spark.rapids.sql.batchSizeBytes', '512M')
# spark.conf.set('spark.rapids.sql.reader.batchSizeBytes', '768M')

In [ ]:
df = read_raw_csv(spark, '../data/titanic/train.csv')

In [ ]:
df.collect()

In [ ]:
!nvidia-smi

## Too Simple
---

You can read on your titanic dataset on GPU